# UKRI Funding and University League Score data cleaning

This notebook reads the required data from the UKRI's and Complete University Guide's websites then cleans and formats it into a dataframe for later use my machine learning models.

- The university league tables are well formatted and able to be phrased with BeautifulSoup fairly easily.
- The UKRI grants data was a bit harder...with each month stored separately with different encodings, inconsistent names and a lack of uniformity in the format of the data. For more details on this see the UKRI grant download section.

In [19]:
import pandas as pd
import requests
import urllib.request
import regex as re
from bs4 import BeautifulSoup

## University League data 2022
The university league table data could be directly pulled from "www.thecompleteuniversityguide.co.uk" using BeautifulSoup. The table is stored as a series of single column lists (all called "col_one") so it was not possible to load the table directly, instead the individual values could be pulled out using bs.find_all with the appropriate filters.

In [28]:
rankings_html = 'https://www.thecompleteuniversityguide.co.uk/league-tables/rankings?tabletype=full-table'
university_ranking_page = requests.get(rankings_html)

In [29]:
soup = BeautifulSoup(university_ranking_page.content, 'html.parser')

In [30]:
institute_names = []
for name in soup.find_all("a", {"class" : "uni_lnk"}):
    institute_names.append(name["data-ga-label"])

In [31]:
institute_scores = []
for item in soup.find_all("div", {"class" : "segtxt"}):
    institute_scores.append(item.findChildren()[0].text)

In [32]:
col_titles = []
for item in soup.find_all("span", {"class" : "hdrtbl"}):
    col_titles.append(item.text)

In [33]:
institute_scores_df = pd.DataFrame(data=np.reshape(institute_scores, (130,11), order='F'), 
                                   index=institute_names, 
                                   columns=col_titles)

In [34]:
institute_scores_df = institute_scores_df.apply(pd.to_numeric, errors='coerce') #convert strings to floats and replace "n/a" with NaN's

In [35]:
institute_scores_df

,Overall score,Entry standards,Student satisfaction,Research quality,Research intensity,Academic services spend,Facilities spend,Degree completion,Student -staff ratio,Graduate prospects – outcomes,Graduate prospects – on track
University of Oxford,1000,200,NaN,3.34,0.87,2842,599,99.1,10.1,90.4,84.7
University of Cambridge,989,205,NaN,3.33,0.95,2718,1043,99.1,11.4,90.0,86.0
"London School of Economics and Political Science, University of London",963,177,3.98,3.35,0.85,2051,853,96.5,12.4,90.6,83.3
University of St Andrews,947,208,4.30,3.13,0.82,2650,746,95.7,11.1,79.9,79.6
Imperial College London,895,194,3.99,3.36,0.92,2982,755,97.5,11.1,95.1,86.7
...,...,...,...,...,...,...,...,...,...,...,...
University of Suffolk,395,110,3.96,NaN,NaN,1961,478,65.0,16.5,74.4,80.9
University of East London,374,97,4.00,2.71,0.23,1105,756,76.5,21.9,56.5,69.6
"Glyndwr University, Wrexham",364,102,4.14,2.15,0.16,1334,559,73.3,21.9,62.9,68.6
Ravensbourne University London,333,113,3.80,NaN,NaN,1163,322,79.7,22.6,70.0,77.5


## UK Research and Innovate (UKRI) funding data for the years 2021
- Total grant data available from www.ukri.org as a list a individual expenses.
- Only the Research Grant Expenditure expense type will be used as this is the money direclty paid to an institution.
 ### Issues
- The data is stored in a Latin encoding, but with a utf-8 byte order mark (BOM), making them incompatible for reading directly with "utf-8-sig", so it is first read using the Latin-1 encoding, then the BOM manually stripped out from the column names.
- Some of the csv's are stoed as "filename.csv.csv"
- The column names are no consistent, neither is the number of columns
- Some of the csv's have text at the top, causing column names to be misplaced and their corresponding data to be unlabeled


In [55]:
UKRI_URL = 'https://www.ukri.org/publications/financial-transparency-data-all-transactions-2021/'
UKRI_html = requests.get(UKRI_URL)
soup = BeautifulSoup(UKRI_html.content, 'html.parser')

In [56]:
monthly_grant_links = []
for name in soup.find_all("a", {"class" : "ukri-publication-csv__link"}):
    monthly_grant_links.append(name["href"])

In [57]:
monthly_grant_links

['https://www.ukri.org/wp-content/uploads/2022/02/UKRI-160222-TotalExpenditureDecember2021.csv.csv',
 'https://www.ukri.org/wp-content/uploads/2022/02/UKRI-020322-TotalExpenditureNovember2021.csv',
 'https://www.ukri.org/wp-content/uploads/2021/11/UKRI-231121-TotalExpenditureOctober2021.csv',
 'https://www.ukri.org/wp-content/uploads/2021/11/UKRI-081121-TotalExpenditureSeptember2021.csv.csv',
 'https://www.ukri.org/wp-content/uploads/2021/11/UKRI-081121-TotalExpenditureAugust2021.csv.csv',
 'https://www.ukri.org/wp-content/uploads/2021/09/UKRI-200921-TotalExpenditureJuly2021.csv',
 'https://www.ukri.org/wp-content/uploads/2021/08/UKRI-020821-TotalExpenditureJune2021.csv',
 'https://www.ukri.org/wp-content/uploads/2021/07/UKRI-060721-Total-Expenditure-May-2021.csv',
 'https://www.ukri.org/wp-content/uploads/2021/06/UKRI-040621-Total-Expenditure-April-2021.csv',
 'https://www.ukri.org/wp-content/uploads/2021/05/UKRI-060521-Total-Expenditure-March-2021.csv',
 'https://www.ukri.org/wp-cont

In [80]:
df_of_monthly_grants={} #reading the csv's using the Latin-1 encoding

for link in monthly_grant_links:
    df_of_monthly_grants[link] = pd.read_csv(link,
                                 encoding='Latin-1', 
                                 dtype=str,
                                 low_memory=False)

In [60]:
for df in df_of_monthly_grants: #removing the BOM
    df_of_monthly_grants[df].columns = df_of_monthly_grants[df].columns.str.replace('ï»¿', '') #removes byte order mark
    df_of_monthly_grants[df].columns = df_of_monthly_grants[df].columns.str.strip()

In [64]:
UKRI_spending = pd.concat(df_of_monthly_grants.values()) #replace strings with floats where possible, or NaN's where not.
UKRI_spending.head()

,Department Family,Entity,Date,Expense Type,Expense Area,Supplier,Transaction Number,Amount,Item Description,Unnamed: 9,...,Unnamed: 11,,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,BEIS,UKRI - AHRC,01/12/2021,T&S UK - Other Charges,Executive Office,Unspecified,99086870,164.48,T&S UK - Other Charges,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Operations,Unspecified,99086145,170.00,Fees (Including College Members),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Health and Environmental Humanities,Unspecified,99086146,340.00,Fees (Including College Members),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Operations,Unspecified,99086147,85.00,Fees (Including College Members),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Health and Environmental Humanities,Unspecified,99086148,340.00,Fees (Including College Members),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
UKRI_spending.describe().transpose()

,count,unique,top,freq
Department Family,306335,1,BEIS,306335
Entity,306335,10,UKRI - MRC,95184
Date,306326,261,15/03/2021,4626
Expense Type,306335,277,Chemicals,45326
Expense Area,305208,1560,Unspecified,53071
Supplier,306001,11587,Unspecified,39198
Transaction Number,306335,189562,99107749,1334
Amount,306335,99279,250.00,7527
Item Description,306335,310,Chemicals,45326
Unnamed: 9,0,0,NaN,NaN


- Columns Unnamed 9, 10 amd 11 contain no non-Nan values and can stripped away
- Columns Unnamed 1-9 mirror the data from the named columns Entity-Item description, so I suspect some of the csv's have a number of dummy columns, with their data stored differently to the other files. Can be confirmed by checking the first row of each file.

In [68]:
UKRI_spending.dropna(how='all', axis=1, inplace=True) #remove columns with only NaN values (Unnamed 9-11)
UKRI_spending.describe().transpose()

,count,unique,top,freq
Department Family,306335,1,BEIS,306335
Entity,306335,10,UKRI - MRC,95184
Date,306326,261,15/03/2021,4626
Expense Type,306335,277,Chemicals,45326
Expense Area,305208,1560,Unspecified,53071
Supplier,306001,11587,Unspecified,39198
Transaction Number,306335,189562,99107749,1334
Amount,306335,99279,250.00,7527
Item Description,306335,310,Chemicals,45326
,86870,7,BEIS,86860


In [ ]:
for df in df_of_monthly_grants: #inspect head of each df from discrepencies
    print(df, df_of_monthly_grants[df].head(1))
# shows different headers for several files

Jan, Aug, Sep all share a different file structure (see below). Opening these files manually shows several lines of text at the top of the files which is changing the column names/positions. The number of lines is not constant.

In [78]:
df_of_monthly_grants["https://www.ukri.org/wp-content/uploads/2021/03/UKRI-040321-Jan21-AllExpenditure.csv"].head(5)

,,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,All Transactions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Department Family,Entity,Date,Expense Type,Expense Area,Supplier,Transaction Number,Amount,Item Description,NaN,NaN,NaN
3,BEIS,UKRI - AHRC,05/01/2021,Research Grant Expenditure,RCUK Global Challenges Research Fund (RCUK GCRF),UKRI - Natural Environment Research Council,98875796,17616.00,Research Grant Expenditure,NaN,NaN,NaN
4,BEIS,UKRI - AHRC,05/01/2021,Research Grant Expenditure,RCUK Global Challenges Research Fund (RCUK GCRF),UKRI - Natural Environment Research Council,98877842,-17616.00,Research Grant Expenditure,NaN,NaN,NaN


# Reimport all csv's but skipping headers in the incorrect files

In [90]:
#reimport the problemtic files with correct headers and replace them in the dict of dataframes
import csv

def find_header(url, nrows, header_signiture):
    """
    Finds the index of the header row.
    url : str()
        url of the csv that you want to phrase
    nrows : int()
        number of rows to check for header (to save importing whole file)
    header_signiture : str()
        string to match to as the header
    """
    
    data = pd.read_csv(link, nrows=nrows,  header=None)
    for row in range(nrows):
        if data.iloc[row][0] == header_signiture:
            return(row)

In [102]:
df_of_monthly_grants_corrected = {}     
        
for link in monthly_grant_links:
    header_row = find_header(link, 20, "Department Family")
    df_of_monthly_grants_corrected[link] = pd.read_csv(link,
                                                     encoding='Latin-1', 
                                                     dtype=str,
                                                     header=[header_row],
                                                     low_memory=False)

for df in df_of_monthly_grants_corrected: #removing the BOM
    df_of_monthly_grants_corrected[df].columns = df_of_monthly_grants_corrected[df].columns.str.replace('ï»¿', '') #removes byte order mark
    df_of_monthly_grants_corrected[df].columns = df_of_monthly_grants_corrected[df].columns.str.strip()    

UKRI_spending = pd.concat(df_of_monthly_grants_corrected.values()) #replace strings with floats where possible, or NaN's where not.
UKRI_spending.dropna(how='all', axis=1, inplace=True) #remove columns with only NaN values (Unnamed 9-11)
UKRI_spending.head()

,Department Family,Entity,Date,Expense Type,Expense Area,Supplier,Transaction Number,Amount,Item Description
0,BEIS,UKRI - AHRC,01/12/2021,T&S UK - Other Charges,Executive Office,Unspecified,99086870,164.48,T&S UK - Other Charges
1,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Operations,Unspecified,99086145,170.00,Fees (Including College Members)
2,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Health and Environmental Humanities,Unspecified,99086146,340.00,Fees (Including College Members)
3,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Operations,Unspecified,99086147,85.00,Fees (Including College Members)
4,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Health and Environmental Humanities,Unspecified,99086148,340.00,Fees (Including College Members)


In [101]:
UKRI_spending.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 685443 entries, 0 to 50849
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Department Family   393195 non-null  object
 1   Entity              393195 non-null  object
 2   Date                393183 non-null  object
 3   Expense Type        393195 non-null  object
 4   Expense Area        391772 non-null  object
 5   Supplier            392861 non-null  object
 6   Transaction Number  393195 non-null  object
 7   Amount              393195 non-null  object
 8   Item Description    393195 non-null  object
dtypes: object(9)
memory usage: 52.3+ MB


In [103]:
UKRI_spending.dropna(how='all', axis=0, inplace=True) #remove row will all NaN values

In [104]:
UKRI_spending.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393196 entries, 0 to 26324
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Department Family   393195 non-null  object
 1   Entity              393195 non-null  object
 2   Date                393183 non-null  object
 3   Expense Type        393195 non-null  object
 4   Expense Area        391772 non-null  object
 5   Supplier            392861 non-null  object
 6   Transaction Number  393195 non-null  object
 7   Amount              393195 non-null  object
 8   Item Description    393195 non-null  object
dtypes: object(9)
memory usage: 30.0+ MB


In [105]:
UKRI_spending.describe().transpose()

,count,unique,top,freq
Department Family,393195,1,BEIS,393195
Entity,393195,10,UKRI - MRC,122918
Date,393183,351,15/03/2021,4626
Expense Type,393195,283,Chemicals,58641
Expense Area,391772,1604,Unspecified,68399
Supplier,392861,12581,Unspecified,47527
Transaction Number,393195,243545,99107749,1334
Amount,393195,117418,250.00,9032
Item Description,393195,316,Chemicals,58641


In [106]:
UKRI_spending.isnull().sum()

Department Family        1
Entity                   1
Date                    13
Expense Type             1
Expense Area          1424
Supplier               335
Transaction Number       1
Amount                   1
Item Description         1
dtype: int64

### Enough entries that we can drop the remaining NaN's

In [108]:
UKRI_spending.dropna(inplace=True)

In [109]:
UKRI_spending.head()

,Department Family,Entity,Date,Expense Type,Expense Area,Supplier,Transaction Number,Amount,Item Description
0,BEIS,UKRI - AHRC,01/12/2021,T&S UK - Other Charges,Executive Office,Unspecified,99086870,164.48,T&S UK - Other Charges
1,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Operations,Unspecified,99086145,170.00,Fees (Including College Members)
2,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Health and Environmental Humanities,Unspecified,99086146,340.00,Fees (Including College Members)
3,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Operations,Unspecified,99086147,85.00,Fees (Including College Members)
4,BEIS,UKRI - AHRC,02/12/2021,Fees (Including College Members),Health and Environmental Humanities,Unspecified,99086148,340.00,Fees (Including College Members)


## Change the dtypes of the different collumns as the import is string only. Involves removing some formatting, and a time traveler...

In [ ]:
UKRI_spending["Date"] = UKRI_spending["Date"].astype('datetime64')
#gives long error about pandas datetime being outside the acceptable range

## Data should be only for the year 2021
- Since some of the data is outside of pandas 64-bit number it must be at least 584 years from the current datetime
- There is in fact a time traveler in the data set, with an expense listed in the year 2929 (see below)
- I'm going to make the assumption that this should have been for 2021 but was entered wrong

In [141]:
for date in UKRI_spending["Date"]:
    if date[-4:] != "2021":
        print(date)

12/08/2929


In [155]:
UKRI_spending.loc[UKRI_spending["Date"]=="12/08/2929", "Date"] = "12/08/2929"

In [156]:
UKRI_spending["Amount"] = UKRI_spending["Amount"].str.replace(",","", regex=False)
UKRI_spending["Amount"] = UKRI_spending["Amount"].str.replace("(","", regex=False)
UKRI_spending["Amount"] = UKRI_spending["Amount"].str.replace(")","", regex=False).astype(float)

# The data sets are now ready for use in the ML part of this project
- saving dataframes as csv's for import into ML projects as webpages may change over time
- Github has a 25MB file size limit so for the UKRI_spending data, only the Research Grand Expenditure is being saved as that is what is used for the models.

In [158]:
UKRI_spending = UKRI_spending[UKRI_spending['Expense Type'] == 'Research Grant Expenditure']
UKRI_spending.to_csv("UKRI_spending_df.csv")
institute_scores_df.to_csv("institute_scores_df.csv")